# Прогноз оттока зарплатного клиента ФЛ

## Команда "Провинция"

### Участники:
##### Сорочайкин Александр Ярославович
##### Родин Владислав Алексеевич
##### Дидуренко Степан Алексеевич
##### Лапшин Дмитрий Сергеевич
##### Иванов Иван Владимирович

# Подключение библиотек

In [64]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, IsolationForest
from sklearn.feature_selection import SelectFromModel, SelectKBest, chi2, mutual_info_classif, RFE
from sklearn.linear_model import Lasso, Ridge, LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
    roc_auc_score,
    roc_curve,
)
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.svm import SVC
import optuna
from catboost import CatBoostClassifier
from scipy.stats import randint
from sklearn.pipeline import make_pipeline
#%pip install catboost
#%pip install optuna

# Первичная обработка данных

In [204]:
df = pd.read_parquet("/home/jupyter/mnt/datasets/DATASET/dataset.parquet")

## Реиндексация

In [205]:
df.reset_index(inplace = True, drop=True)
df

,id,target,sample_ml_new,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15,feature16,feature17,feature18,feature19,feature20,feature21,feature22,feature23,feature24,feature25,feature26,feature27,feature28,feature29,feature30,feature31,feature32,feature33,feature34,feature35,feature36,feature37,...,feature1037,feature1038,feature1039,feature1040,feature1041,feature1042,feature1043,feature1044,feature1045,feature1046,feature1047,feature1048,feature1049,feature1050,feature1051,feature1052,feature1053,feature1054,feature1055,feature1056,feature1057,feature1058,feature1059,feature1060,feature1061,feature1062,feature1063,feature1064,feature1065,feature1066,feature1067,feature1068,feature1069,feature1070,feature1071,feature1072,feature1073,feature1074,feature1075,feature1076
0,0,0,1,1761,1759,85,105469,191,46,0,6,95,0,0,20,191,1,6,50,60,7,248,248,347,91601,9,173,1722,5,8,9,174,1738,633,1545,50524,51771,16130,3652,85495,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12045,12107,12510,21126,28913,48985,84264,12045,12107,12510,21126,28913,48985,84264
1,1,0,1,1761,1759,120,105610,144,71,0,135,135,0,0,187,144,1,135,43,101,11,18,18,58,36890,9,184,1722,5,8,9,185,1738,633,1637,50524,51771,18999,3652,85495,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,0,1,890,1759,141,105227,191,11,0,0,0,0,0,187,191,1,0,0,0,28,248,248,347,140184,9,248,1722,5,8,9,250,1738,633,1756,50524,51771,65748,3652,85495,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,33735,31242,12691,48682,55949,70006,113317,33735,31242,12691,48682,55949,70006,113317
3,5,0,1,1599,966,30,102441,191,8,0,0,0,0,0,187,191,1,0,0,0,28,248,248,347,140184,9,248,1722,5,8,9,250,1738,633,1756,50524,51771,65748,3652,85495,...,0,12329,12291,12291,12291,101,0,0,114,118,118,118,0,0,0,0,0,0,0,5220,0,0,5810,5890,5890,5890,0,0,0,0,0,6872,8530,0,0,0,0,0,6872,8530
4,6,0,1,1761,1759,85,104006,191,2,0,0,46,0,0,2,191,1,0,1,21,3,248,248,347,140184,9,248,1722,5,8,9,250,1738,633,1756,16988,51771,65748,3652,0,...,0,26719,26759,26759,26759,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7642,0,0,0,0,0,0,7642
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519610,693036,0,1,1761,1759,141,410,191,2,125,290,298,176,121,187,191,227,282,243,275,28,248,248,347,140184,9,248,1722,5,8,9,250,1738,633,1756,50524,51771,65748,3652,85495,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
519611,693038,0,1,1761,1759,0,170,191,1,125,290,298,176,121,187,191,227,282,243,275,28,248,248,347,140184,9,248,1722,5,8,9,250,1738,633,1756,50524,51771,65748,3652,85495,...,19232,0,0,0,0,0,0,3901,0,0,0,0,0,0,1638,0,0,0,0,0,0,10091,0,0,0,0,0,0,51714,0,0,0,0,0,0,51714,0,0,0,0
519612,693039,0,1,1761,1759,0,168,191,1,125,290,298,176,121,187,191,227,282,243,275,28,248,248,347,140184,9,248,1722,5,8,9,250,1738,633,1756,50524,51771,65748,3652,85495,...,19232,0,0,0,0,0,0,3901,0,0,0,0,0,0,1638,0,0,0,0,0,0,10091,0,0,0,0,0,0,51714,0,0,0,0,0,0,51714,0,0,0,0
519613,693040,0,1,1761,1759,141,74,191,1,125,290,298,176,121,187,191,227,282,243,275,28,248,248,347,140184,9,248,1722,5,8,9,250,1738,633,1756,50524,51771,65748,3652,85495,...,19232,0,0,0,0,0,0,3901,0,0,0,0,0,0,1638,0,0,0,0,0,0,10091,0,0,0,0,0,0,51714,0,0,0,0,0,0,51714,0,0,0,0


## Выявляем признаки, у которых процент Null значений больше порогового

In [206]:
def check_nans_features(df, lower_bound):
    nans_features = []
    for column in df.columns:
        if df[column].isnull().sum() / len(df[column]) > lower_bound:
            nans_features.append(column)
    return nans_features

## Выявляем признаки, у которых только одно уникальное значение

In [207]:
def check_empty_features(df):
    empty_features = []
    for column in df.columns:
        if df[column].nunique() == 1:
            empty_features.append(column)
    return empty_features

## Удаляем найденные признаки

In [208]:
nans_features = check_nans_features(df, 0.5)
empty_features = check_empty_features(df)


df.drop(columns = nans_features, inplace=True)
df.drop(columns=empty_features, inplace=True)
df.shape

(519615, 988)

## Удаляем неуникальные признаки и столбцы

In [209]:
df.drop_duplicates(inplace=True, ignore_index=True)
df.shape

(519615, 988)

## Находим категориальные признаки и столбцы

In [210]:
def check_categorical_features(df):
    cool_features = []
    for column in df.columns:
        if df[column].nunique() == 2:
            df[column] = df[column].apply(lambda x: 1 if x != 0 else 0)
            cool_features.append(column)
    return cool_features

## Выносим target в отдельную переменную, удаляем странный столбец, найденный в процессе работы

In [212]:
target = df["target"]
df.drop(["target"], axis = 1, inplace = True)

In [213]:
df.drop(["feature774"], axis=1, inplace=True)

## Удаляем найденные признаки и отделяем числовые от категориальных (чекаю шейпы только для локальной валидации)

In [214]:
print(df.shape)

categorical_features = check_categorical_features(df)
df_categorical = df[categorical_features]
df_numerical = df.drop(columns=categorical_features)


print(df_categorical.shape)
print(df_numerical.shape)

(519615, 986)
(519615, 146)
(519615, 840)


## Заполнение медианами пропусков в числовых фичах + конкатенация в один датафрейм

In [215]:
df_numerical.fillna(value=df_numerical.median(), inplace=True)

In [216]:
df = pd.concat([df_numerical, target, df_categorical], axis = 1)

In [16]:
# df = df.sample(frac=1, axis=1) - возможно столбцы стоит шафлить

## Обработка выбросов

In [19]:
def anomal_index(df):
    # Настройка гиперпараметров с использованием RandomizedSearchCV
    param_dist = {'n_estimators': randint(50, 200),
                  'max_samples': ['auto', 0.5, 0.75],
                  'max_features': [1.0, 0.5, 0.75]}
    
    # Указание ROC-AUC в качестве основной метрики качества
    clf = RandomizedSearchCV(IsolationForest(random_state=42), param_dist, n_iter=10, cv=3, scoring='roc_auc', n_jobs=-1)
    clf.fit(df)
    
    # Поиск оптимальных гиперпараметров
    best_params = clf.best_params_
    
    # Создание экземпляра IsolationForest с оптимальными параметрами
    best_clf = IsolationForest(**best_params, random_state=42)
    
    # Обучение модели на данных
    best_clf.fit(df)
    
    # Предсказание выбросов
    return best_clf.predict(df) == 1

In [217]:
print(df.shape)

not_anomal_index = anomal_index(df)
df = df[not_anomal_index]

print(df.shape)

(519615, 987)
(518456, 987)


## Переиндексация

In [218]:
df.reset_index(inplace = True, drop = True)

print(df.shape)

(518456, 987)


## Разбиваем на категориальные и числовые признаки

In [219]:
target = df["target"]
df.drop(columns=["target"], inplace=True)
df_categorical = df[categorical_features]
df_numerical = df.drop(columns=categorical_features)

print(df_categorical.shape)
print(df_numerical.shape)
print(target.shape)

(518456, 146)
(518456, 840)
(518456,)


# Обработка категориальных признаков

## Выявление важных категориальных признаков с помощью моделей ML

In [221]:
df_1 = df_categorical[target == 1]
df_1_r = df_1.sample(17500)
df_1_r

,feature647,feature707,feature659,feature670,feature790,feature668,feature396,feature618,feature435,feature709,feature434,feature711,feature649,feature567,feature741,feature695,feature671,feature693,feature691,feature795,feature630,feature787,feature619,feature631,feature675,feature732,feature726,feature544,feature719,feature245,feature688,feature400,feature628,feature698,feature536,feature729,feature405,feature477,feature684,feature652,...,feature464,feature689,feature385,feature664,feature793,feature388,feature597,feature561,feature644,feature242,feature384,feature476,feature654,feature646,feature791,feature537,feature465,feature445,feature720,feature673,feature687,feature655,feature235,feature640,feature569,feature667,feature645,feature463,feature624,feature703,feature662,feature633,feature401,feature627,feature792,feature713,feature676,feature599,feature794,feature386
151572,1,0,0,0,1,1,1,0,1,0,1,0,1,0,0,0,0,1,1,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,1,...,0,1,1,1,0,1,1,1,1,0,1,1,1,1,1,1,0,1,0,1,1,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,1,0,1,1
44207,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,1,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0
77890,0,0,1,0,1,1,0,0,0,0,0,0,1,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,...,0,1,1,1,0,1,1,1,1,0,0,0,1,0,1,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,1,1
307007,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,...,0,1,1,1,0,1,0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0
418373,0,0,0,0,1,0,1,0,1,0,1,0,1,0,0,1,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,1,0,1,0,1,1,1,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295315,0,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,...,1,1,1,0,0,1,0,1,1,1,0,0,1,0,1,1,1,0,0,0,0,0,1,0,1,0,1,1,0,0,0,0,0,0,0,1,0,0,1,0
385784,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,...,0,1,0,1,1,1,0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0
350100,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,...,0,1,1,1,0,1,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0
217312,1,0,1,0,1,1,1,0,1,0,0,0,1,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,...,0,1,0,1,0,1,1,1,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,1,1,0,1,1


In [222]:
df_0 = df_categorical[target == 0]
df_0_r = df_0.sample(17500)
df_0_r

,feature647,feature707,feature659,feature670,feature790,feature668,feature396,feature618,feature435,feature709,feature434,feature711,feature649,feature567,feature741,feature695,feature671,feature693,feature691,feature795,feature630,feature787,feature619,feature631,feature675,feature732,feature726,feature544,feature719,feature245,feature688,feature400,feature628,feature698,feature536,feature729,feature405,feature477,feature684,feature652,...,feature464,feature689,feature385,feature664,feature793,feature388,feature597,feature561,feature644,feature242,feature384,feature476,feature654,feature646,feature791,feature537,feature465,feature445,feature720,feature673,feature687,feature655,feature235,feature640,feature569,feature667,feature645,feature463,feature624,feature703,feature662,feature633,feature401,feature627,feature792,feature713,feature676,feature599,feature794,feature386
279882,1,0,0,0,1,1,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,...,0,1,0,1,1,1,0,1,1,1,0,1,1,1,1,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0
110236,1,0,0,0,1,1,1,0,1,0,1,0,1,0,0,0,0,0,1,1,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,1,0,1,...,0,1,1,1,0,1,1,1,1,0,0,1,1,1,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,1,0,1,0
28256,0,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,...,0,1,0,1,1,1,0,1,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,1,1
109487,0,0,1,0,1,1,1,0,0,0,1,0,1,0,0,1,0,0,1,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,...,0,1,0,1,0,1,1,1,1,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0
515576,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,1,0,1,1,1,0,1,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7358,0,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,...,1,1,0,1,1,1,0,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,1,1,1,0,1,0
504804,0,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,...,0,1,1,1,0,1,0,1,1,1,0,0,1,0,1,1,0,0,0,0,0,1,1,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0
404878,0,0,0,0,1,1,0,0,1,0,0,0,1,0,0,1,0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,1,1,0,1,0,1,1,1,1,0,0,0,1,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0
101723,1,0,0,0,1,1,1,0,1,0,0,0,1,0,0,1,0,0,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,...,0,1,1,1,0,1,1,1,1,0,1,0,1,1,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0


In [223]:
df_res_res = pd.concat([df_1_r, df_0_r])

# Перемешивание строк в DataFrame
shuffled_data = df_res_res.sample(
    frac=1, random_state=42
)  
shuffled_data

,feature647,feature707,feature659,feature670,feature790,feature668,feature396,feature618,feature435,feature709,feature434,feature711,feature649,feature567,feature741,feature695,feature671,feature693,feature691,feature795,feature630,feature787,feature619,feature631,feature675,feature732,feature726,feature544,feature719,feature245,feature688,feature400,feature628,feature698,feature536,feature729,feature405,feature477,feature684,feature652,...,feature464,feature689,feature385,feature664,feature793,feature388,feature597,feature561,feature644,feature242,feature384,feature476,feature654,feature646,feature791,feature537,feature465,feature445,feature720,feature673,feature687,feature655,feature235,feature640,feature569,feature667,feature645,feature463,feature624,feature703,feature662,feature633,feature401,feature627,feature792,feature713,feature676,feature599,feature794,feature386
303655,0,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,...,0,1,0,1,0,1,1,1,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0
287276,0,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,1,0,1,1,0,1,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0
168321,1,0,0,0,1,1,1,0,1,0,1,0,1,0,0,1,0,0,1,1,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,...,1,1,0,1,1,1,1,1,1,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,1,1,1,0,1,0
498865,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,0,1,0,1,1,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0
430602,0,0,0,0,1,0,1,0,1,0,1,0,1,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,1,0,1,0,1,1,1,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88856,1,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,...,0,1,0,1,0,1,0,1,1,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,1,0,0,0
82887,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,...,0,1,1,1,0,1,0,1,1,0,0,1,0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0
283644,0,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,...,0,1,1,1,0,1,0,1,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0
174901,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,1,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,...,0,1,0,1,0,1,1,1,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0


In [224]:
target_indexes = shuffled_data.index

In [225]:
# Загрузка данных (X - матрица признаков, y - вектор целевой переменной)
X = shuffled_data
y = target.loc[target_indexes].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
# Создаем пайплайн для отбора признаков и обучения модели для каждой модели
models = {
    'SVM': SVC(kernel="linear"),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier()
}

selected_features = {}

for name, model in models.items():
    pipe = make_pipeline(SelectFromModel(model), model)
    # Выполняем кросс-валидацию для оценки важности признаков с метрикой ROC-AUC
    scores = cross_val_score(pipe, X_train, y_train, cv=5, scoring='roc_auc')
    pipe.fit(X_train, y_train)
    selected_features[name] = X_train.columns[pipe.named_steps['selectfrommodel'].get_support()]

    print(f"Модель: {name}")
    print(f"Средняя оценка ROC-AUC кросс-валидации: {scores.mean()}")

# Объединяем отобранные признаки из всех моделей
all_selected_features = set()
for features in selected_features.values():
    all_selected_features.update(features)

print("Отобранные признаки из всех моделей:", all_selected_features)


Модель: SVM
Средняя оценка ROC-AUC кросс-валидации: 0.641655281758618
Модель: Random Forest
Средняя оценка ROC-AUC кросс-валидации: 0.6066033909535749
Модель: Gradient Boosting
Средняя оценка ROC-AUC кросс-валидации: 0.6610802249850142
Отобранные признаки из всех моделей: {'feature647', 'feature707', 'feature659', 'feature670', 'feature790', 'feature668', 'feature396', 'feature618', 'feature435', 'feature709', 'feature434', 'feature711', 'feature649', 'feature567', 'feature741', 'feature695', 'feature671', 'feature693', 'feature691', 'feature795', 'feature630', 'feature787', 'feature619', 'feature631', 'feature675', 'feature732', 'feature726', 'feature544', 'feature719', 'feature245', 'feature688', 'feature400', 'feature628', 'feature698', 'feature536', 'feature729', 'feature405', 'feature477', 'feature684', 'feature652', 'feature443', 'feature755', 'feature680', 'feature639', 'feature545', 'feature588', 'feature437', 'feature788', 'feature444', 'feature464', 'feature689', 'feature385'

In [226]:
df_categorical = df.loc[:, all_selected_features]
df_categorical

<ipython-input-226-66efa792323c>:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  df_categorical = df.loc[:, all_selected_features]


,feature647,feature707,feature659,feature670,feature790,feature668,feature396,feature618,feature435,feature709,feature434,feature711,feature649,feature567,feature741,feature695,feature671,feature693,feature691,feature795,feature630,feature787,feature619,feature631,feature675,feature732,feature726,feature544,feature719,feature245,feature688,feature400,feature628,feature698,feature536,feature729,feature405,feature477,feature684,feature652,...,feature464,feature689,feature385,feature664,feature793,feature388,feature597,feature561,feature644,feature242,feature384,feature476,feature654,feature646,feature791,feature537,feature465,feature445,feature720,feature673,feature687,feature655,feature235,feature640,feature569,feature667,feature645,feature463,feature624,feature703,feature662,feature633,feature401,feature627,feature792,feature713,feature676,feature599,feature794,feature386
0,1,0,0,0,1,1,1,0,1,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,...,0,1,0,1,0,1,1,1,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0
1,1,0,0,0,1,1,1,0,1,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,1,...,0,1,1,1,0,1,1,1,1,0,1,0,1,0,1,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0
2,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,1,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0
3,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,...,0,1,0,1,0,1,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0
4,0,0,0,0,1,0,1,0,1,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,...,0,1,0,0,1,1,0,1,1,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518451,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,...,0,1,0,1,1,1,0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
518452,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,...,0,1,0,1,1,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0
518453,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,...,0,1,0,0,1,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0
518454,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,...,0,1,0,1,1,1,0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0


## Отбор на слишком несбалансированные и не являющиеся важными категориальные фичи

In [233]:
# функция отбирает столбцы, в которых значения таблицы только 1 или 0
# отбирает из них колонки с дисбалансом класса
# и возвращает массив кортежей (название колонки, минимальное по количеству значение)
# 
# 
# функция принимает на вход df - исходный DataFrame,
# lower_bound - нижняя граница,
# upper_bound - верхняя граница

def disbalanced_categories(df, lower_bound, upper_bound): # функция отбирает столбцы, в которых значения
    # print(df)
    no_balanced_categories = []                           # таблицы только 1 или 0
    for column in df.columns:                             # 
        if(len(df[column].unique()) == 2 and not ((df[column].unique()[1]>1))):
            values = df[column].value_counts()
            cnt_min = 0
            val = 0
            if values.loc[0] < values.loc[1]:
                cnt_min = values.loc[0]
                val = 0
            else:
                cnt_min = values.loc[1]
                val = 1
            if lower_bound <= cnt_min <= upper_bound:
                no_balanced_categories.append((column, val))
    return no_balanced_categories

# отбираем столбцы у которых отношение
# минимиального несбалансированного значения совпадающего с target == 1
# на кол-во минимального несбалансированного значения
# меньше 0.5
# 
# функция принимает на вход df - исходный DataFrame,
# disbalanced_categories - название столбцов с несбалансированными категориями

def target_match(df, disbalanced_categories, target):
    droped_columns = []
    for column, value in disbalanced_categories:
        s = df[(df[column] == value) & (target == 1)][column].sum()
        al = df[column].value_counts()[value]
        if al == 0 or s/al <= 0.5:
            droped_columns.append(column)
    return droped_columns

In [234]:
categories_to_drop = disbalanced_categories(df_categorical, 0, 700)
df_disbalanced = disbalanced_categories(df_categorical, 700, 1800)
excess_categorical = target_match(df_categorical, df_disbalanced, target)

features_to_drop = [column for column, values in categories_to_drop]
features_to_drop += excess_categorical

df_categorical.drop(columns = features_to_drop, inplace = True)
print(df_categorical.shape)

(518456, 50)


In [235]:
print(df_categorical.shape)
print(df_numerical.shape)
print(target.shape)

(518456, 50)
(518456, 840)
(518456,)


In [236]:
df_numerical

,id,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15,feature16,feature17,feature18,feature19,feature20,feature21,feature22,feature23,feature24,feature25,feature26,feature27,feature28,feature29,feature30,feature31,feature32,feature33,feature34,feature35,feature36,feature37,feature38,feature39,...,feature1004,feature1035,feature1036,feature1038,feature1039,feature1040,feature1041,feature1042,feature1043,feature1045,feature1046,feature1047,feature1048,feature1049,feature1050,feature1051,feature1052,feature1053,feature1054,feature1055,feature1056,feature1057,feature1059,feature1060,feature1061,feature1062,feature1063,feature1064,feature1065,feature1066,feature1067,feature1068,feature1069,feature1070,feature1071,feature1072,feature1073,feature1074,feature1075,feature1076
0,0,1761,1759,85,105469,191,46,0,6,95,0,0,20,191,1,6,50,60,7,248,248,347,91601,9,173,1722,5,8,9,174,1738,633,1545,50524,51771,16130,3652,85495,14172,11665,...,14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12045,12107,12510,21126,28913,48985,84264,12045,12107,12510,21126,28913,48985,84264
1,1,1761,1759,120,105610,144,71,0,135,135,0,0,187,144,1,135,43,101,11,18,18,58,36890,9,184,1722,5,8,9,185,1738,633,1637,50524,51771,18999,3652,85495,14172,11665,...,22,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,890,1759,141,105227,191,11,0,0,0,0,0,187,191,1,0,0,0,28,248,248,347,140184,9,248,1722,5,8,9,250,1738,633,1756,50524,51771,65748,3652,85495,14172,11665,...,40,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,33735,31242,12691,48682,55949,70006,113317,33735,31242,12691,48682,55949,70006,113317
3,5,1599,966,30,102441,191,8,0,0,0,0,0,187,191,1,0,0,0,28,248,248,347,140184,9,248,1722,5,8,9,250,1738,633,1756,50524,51771,65748,3652,85495,81,10,...,3,12436,0,12329,12291,12291,12291,101,0,114,118,118,118,0,0,0,0,0,0,0,5220,0,5810,5890,5890,5890,0,0,0,0,0,6872,8530,0,0,0,0,0,6872,8530
4,6,1761,1759,85,104006,191,2,0,0,46,0,0,2,191,1,0,1,21,3,248,248,347,140184,9,248,1722,5,8,9,250,1738,633,1756,16988,51771,65748,3652,0,14172,11665,...,12,27032,0,26719,26759,26759,26759,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7642,0,0,0,0,0,0,7642
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518451,693036,1761,1759,141,410,191,2,125,290,298,176,121,187,191,227,282,243,275,28,248,248,347,140184,9,248,1722,5,8,9,250,1738,633,1756,50524,51771,65748,3652,85495,14172,11665,...,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
518452,693038,1761,1759,0,170,191,1,125,290,298,176,121,187,191,227,282,243,275,28,248,248,347,140184,9,248,1722,5,8,9,250,1738,633,1756,50524,51771,65748,3652,85495,14172,11665,...,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1638,0,0,0,0,0,0,0,0,0,0,0,0,51714,0,0,0,0,0,0,51714,0,0,0,0
518453,693039,1761,1759,0,168,191,1,125,290,298,176,121,187,191,227,282,243,275,28,248,248,347,140184,9,248,1722,5,8,9,250,1738,633,1756,50524,51771,65748,3652,85495,14172,11665,...,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1638,0,0,0,0,0,0,0,0,0,0,0,0,51714,0,0,0,0,0,0,51714,0,0,0,0
518454,693040,1761,1759,141,74,191,1,125,290,298,176,121,187,191,227,282,243,275,28,248,248,347,140184,9,248,1722,5,8,9,250,1738,633,1756,50524,51771,65748,3652,85495,14172,11665,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1638,0,0,0,0,0,0,0,0,0,0,0,0,51714,0,0,0,0,0,0,51714,0,0,0,0


# Обработка числовых признаков

## Фильтрация фичей с помощью статистического метода Хи - квадрат

#### У нас есть один столбцец с отрицательными значениями. статистические методы плохо работают с этим, откинем его

In [237]:
df_numerical.drop('feature712', axis = 1, inplace = True)

In [238]:
df_numerical

,id,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15,feature16,feature17,feature18,feature19,feature20,feature21,feature22,feature23,feature24,feature25,feature26,feature27,feature28,feature29,feature30,feature31,feature32,feature33,feature34,feature35,feature36,feature37,feature38,feature39,...,feature1004,feature1035,feature1036,feature1038,feature1039,feature1040,feature1041,feature1042,feature1043,feature1045,feature1046,feature1047,feature1048,feature1049,feature1050,feature1051,feature1052,feature1053,feature1054,feature1055,feature1056,feature1057,feature1059,feature1060,feature1061,feature1062,feature1063,feature1064,feature1065,feature1066,feature1067,feature1068,feature1069,feature1070,feature1071,feature1072,feature1073,feature1074,feature1075,feature1076
0,0,1761,1759,85,105469,191,46,0,6,95,0,0,20,191,1,6,50,60,7,248,248,347,91601,9,173,1722,5,8,9,174,1738,633,1545,50524,51771,16130,3652,85495,14172,11665,...,14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12045,12107,12510,21126,28913,48985,84264,12045,12107,12510,21126,28913,48985,84264
1,1,1761,1759,120,105610,144,71,0,135,135,0,0,187,144,1,135,43,101,11,18,18,58,36890,9,184,1722,5,8,9,185,1738,633,1637,50524,51771,18999,3652,85495,14172,11665,...,22,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,890,1759,141,105227,191,11,0,0,0,0,0,187,191,1,0,0,0,28,248,248,347,140184,9,248,1722,5,8,9,250,1738,633,1756,50524,51771,65748,3652,85495,14172,11665,...,40,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,33735,31242,12691,48682,55949,70006,113317,33735,31242,12691,48682,55949,70006,113317
3,5,1599,966,30,102441,191,8,0,0,0,0,0,187,191,1,0,0,0,28,248,248,347,140184,9,248,1722,5,8,9,250,1738,633,1756,50524,51771,65748,3652,85495,81,10,...,3,12436,0,12329,12291,12291,12291,101,0,114,118,118,118,0,0,0,0,0,0,0,5220,0,5810,5890,5890,5890,0,0,0,0,0,6872,8530,0,0,0,0,0,6872,8530
4,6,1761,1759,85,104006,191,2,0,0,46,0,0,2,191,1,0,1,21,3,248,248,347,140184,9,248,1722,5,8,9,250,1738,633,1756,16988,51771,65748,3652,0,14172,11665,...,12,27032,0,26719,26759,26759,26759,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7642,0,0,0,0,0,0,7642
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518451,693036,1761,1759,141,410,191,2,125,290,298,176,121,187,191,227,282,243,275,28,248,248,347,140184,9,248,1722,5,8,9,250,1738,633,1756,50524,51771,65748,3652,85495,14172,11665,...,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
518452,693038,1761,1759,0,170,191,1,125,290,298,176,121,187,191,227,282,243,275,28,248,248,347,140184,9,248,1722,5,8,9,250,1738,633,1756,50524,51771,65748,3652,85495,14172,11665,...,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1638,0,0,0,0,0,0,0,0,0,0,0,0,51714,0,0,0,0,0,0,51714,0,0,0,0
518453,693039,1761,1759,0,168,191,1,125,290,298,176,121,187,191,227,282,243,275,28,248,248,347,140184,9,248,1722,5,8,9,250,1738,633,1756,50524,51771,65748,3652,85495,14172,11665,...,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1638,0,0,0,0,0,0,0,0,0,0,0,0,51714,0,0,0,0,0,0,51714,0,0,0,0
518454,693040,1761,1759,141,74,191,1,125,290,298,176,121,187,191,227,282,243,275,28,248,248,347,140184,9,248,1722,5,8,9,250,1738,633,1756,50524,51771,65748,3652,85495,14172,11665,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1638,0,0,0,0,0,0,0,0,0,0,0,0,51714,0,0,0,0,0,0,51714,0,0,0,0


In [239]:
# Метод фильтрации фичей по Хи-квадрату
# Функция, которая фидльтуркет данные по методу хи-квадрат 
# источник с описание метода (https://habr.com/ru/articles/264915/)
# 
# Метод не всегда убирает только нужные нам фичи,
# пример когда это не работает есть в источнике
# 
# 
# Функция принимает на вход df - наш датафрей с цифровыми фичами,
# target - наша целевая переменная, 
# n - кол-во фичей которые мы хотим оставить

def chi2_k_select(df, target, n):
    select = SelectKBest(chi2, k=n)
    X_new = select.fit_transform(df, target)
    selected_features_indices = select.get_support(indices=True)
    return list(df.columns[selected_features_indices])

In [240]:
n = 650 # нужно выбрать сколько фичей мы отберем 
chi2_feature = chi2_k_select(df_numerical, target, n)
df_numerical = df_numerical[chi2_feature]
df_numerical.shape

(518456, 650)

## Корелляция. Выявление признаков, объясняющих наибольшее число других

In [241]:
# Функция находит признаки, которые оьъясняют наибольшее кол-во других признаков
# Возвращает список названий признаков и множество объясненных
# df - DataFrame
# corr_threshold - коэффициент корреляции, начиная с которого один признак объясняет другой
# new_features_threshold - если какой-то признак объясняет new_features_threshold новых признаков или больше, мы его включаем
def most_correlated_features_v2(df, corr_threshold=0.995, new_features_threshold=1):
    df_corr = pd.DataFrame(
        np.abs(np.corrcoef(df.values, rowvar=False)),
        columns=df.columns,  # считаем матрицу корреляции через np.corrcoef (так быстрее)
    )
    upper = df_corr.where(
        np.triu(np.ones(df_corr.shape), k=1).astype(bool)
    )  # убираем одну половину

    labels_and_list_of_correlating_features = [  # список пар (признак; множество тех, которые он объясняет)
        (
            label,  # объясняющий признак
            set(
                upper.columns[i]
                for i, e in enumerate(upper[label])
                if e
                > corr_threshold  # включаем признак, если позволяет коэффициент корреляции
            ),
        )
        for label in upper.columns
    ]
    labels_and_list_of_correlating_features.sort(
        key=lambda x: len(x[1]), reverse=True
    )  # сортируем по кол-ву объясненных
    labels_result = list()  # результат - список подходящих признаков
    explained = set()  # множество подходящих признаков
    for label, correlating_features in labels_and_list_of_correlating_features:
        if label in explained:  # если уже объяснили признак
            continue
        elif (
            len(explained) == 0  # только начали
            or len(correlating_features)
            == 0  # признак, который не зависит ни от какого другого
            or len(explained.union(correlating_features))
            - len(explained)  # считаем, сколько объясненных признаков добавится
            >= new_features_threshold
        ):
            labels_result.append(label)
            explained = explained.union(correlating_features)
            explained.add(label)

    return labels_result

In [242]:
df_numerical = df_numerical[most_correlated_features_v2(df_numerical)]

# Обучение моделей

## Предобработка данных для обучения

In [243]:
# Объединение числовых и категориальных данных
df_train = pd.concat([df_numerical, df_categorical], axis=1)
df_train.drop("id", axis = 1, inplace = True)

### Подготовим тестовую часть. Загрузим тестовую выборку и sample_submission

In [244]:
submission = pd.read_csv("sample_submission.csv")
test_data = pd.read_parquet("test_sber.parquet")
test_data = test_data.loc[:, df_train.columns]
df_train["target"] = target
print(test_data.shape)
print(df_train.shape)

(173433, 553)
(518456, 554)


### Создание датасетов с количеством данных, подходящим для обучения

##### Датасет с балансировкой классов

In [247]:
df_1_n = df_train[df_train['target'] == 1]
df_1_r_n = df_1_n
df_1_r_n
df_0_n = df_train[df_train['target'] == 0]
df_0_r_n = df_0_n.sample(20000)
df_0_r_n
df_train_not_balanced = pd.concat([df_1_r_n, df_0_r_n]) # Проверить, что сконкатило нормально и перемешать
df_train_not_balanced

,feature615,feature591,feature612,feature742,feature747,feature905,feature1041,feature1048,feature1062,feature11,feature21,feature251,feature327,feature378,feature593,feature635,feature858,feature860,feature863,feature909,feature911,feature15,feature29,feature50,feature51,feature90,feature132,feature134,feature207,feature211,feature213,feature221,feature223,feature228,feature256,feature258,feature266,feature267,feature313,feature314,...,feature235,feature569,feature713,feature794,feature647,feature668,feature435,feature434,feature695,feature795,feature787,feature732,feature726,feature400,feature477,feature755,feature545,feature437,feature788,feature464,feature385,feature664,feature793,feature597,feature242,feature384,feature476,feature654,feature646,feature537,feature445,feature401,feature792,feature676,feature671,feature536,feature684,feature588,feature640,target
11,1627,10,36,27,0,2550,0,0,0,121,347,71969,8129,2,6,0,5392,5386,2021,6364,1667,282,250,11031,5136,165865,124171,144,21581,119808,139964,89207,98464,0,12949,170158,224177,6679,27,117770,...,1,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
31,0,0,0,27,0,2550,0,0,0,0,347,0,8129,24,0,0,5392,5386,2429,6244,147,0,250,375109,377641,0,124171,9746,402357,71572,79670,20517,28466,0,0,0,0,0,7158,436285,...,0,1,1,0,0,1,1,0,0,1,0,0,0,0,1,1,0,0,0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,1
52,1627,10,36,27,27,2550,24204,0,27097,121,347,71969,8129,21,6,0,5392,5386,166,2987,166,282,250,258011,276136,165865,124171,1645,283522,256847,290209,89207,98464,0,12949,170158,224177,6679,550,1431,...,1,0,1,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
109,0,1,2,27,0,2550,0,250,0,0,203,0,8129,41,0,0,458,458,705,8341,458,133,250,122953,160276,50573,74351,127,110889,263680,295567,89207,98464,0,0,134709,129534,0,20,234577,...,0,0,1,1,1,1,1,1,1,1,0,0,0,0,1,1,0,1,0,0,1,1,0,1,0,1,1,1,1,0,1,0,0,1,0,0,0,0,0,1
169,71,0,1,0,0,2550,0,0,0,0,283,0,8129,16,0,0,265,265,703,3722,187,0,250,85423,222018,77951,82148,87,92946,107602,226031,89207,98464,0,0,68747,63209,0,11,14010,...,0,0,1,0,1,0,1,1,1,1,0,1,0,0,1,1,0,0,0,1,0,1,0,1,0,0,1,1,1,1,0,1,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353429,1627,10,36,0,2,2550,28087,14679,0,121,255,71969,8129,61,6,0,2045,2044,5353,5359,2045,282,250,551184,633269,165506,87299,9746,661943,589943,615829,89207,98464,0,12949,170158,224177,6679,7158,494843,...,1,0,1,1,1,0,0,0,0,1,0,0,0,1,0,1,0,1,0,0,1,1,0,0,1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0
367344,0,0,2,27,0,2550,0,0,0,0,4,0,8129,37,0,0,3931,3926,2212,4981,56,0,250,325041,343046,160596,116134,9746,404441,511142,594102,5478,10046,0,0,164235,178187,0,7158,290952,...,0,1,1,1,1,1,1,0,1,1,0,0,0,0,1,1,1,0,0,0,1,1,1,1,0,1,0,1,1,0,1,0,1,1,0,0,0,0,0,0
2427,1627,10,36,27,0,2550,2761,0,0,121,347,71969,8129,28,6,0,5392,5386,946,9234,946,282,250,141015,97538,0,124171,9746,194848,177925,238356,89207,98464,0,12949,170158,224177,6679,7158,86000,...,1,0,1,1,0,0,1,0,0,1,0,0,0,0,1,1,0,1,0,1,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0
174034,0,0,2,0,0,2550,32410,0,36330,0,347,0,8129,303,0,0,5392,5386,289,6817,79,0,250,547037,629680,0,124171,9746,655686,161956,301152,89207,98464,0,0,99877,92949,0,7158,209587,...,0,0,1,0,1,1,0,1,1,1,0,1,0,0,0,1,1,0,0,0,0,1,1,1,0,0,1,1,0,1,0,0,0,1,0,0,0,0,0,0


In [248]:
# Перемешивание строк в DataFrame
shuffled_data_not_balanced = df_train_not_balanced.sample(
    frac=1, random_state=42
)  # frac=1 означает, что мы перемешиваем все строки
shuffled_data_not_balanced

,feature615,feature591,feature612,feature742,feature747,feature905,feature1041,feature1048,feature1062,feature11,feature21,feature251,feature327,feature378,feature593,feature635,feature858,feature860,feature863,feature909,feature911,feature15,feature29,feature50,feature51,feature90,feature132,feature134,feature207,feature211,feature213,feature221,feature223,feature228,feature256,feature258,feature266,feature267,feature313,feature314,...,feature235,feature569,feature713,feature794,feature647,feature668,feature435,feature434,feature695,feature795,feature787,feature732,feature726,feature400,feature477,feature755,feature545,feature437,feature788,feature464,feature385,feature664,feature793,feature597,feature242,feature384,feature476,feature654,feature646,feature537,feature445,feature401,feature792,feature676,feature671,feature536,feature684,feature588,feature640,target
31638,1627,10,36,27,0,2550,0,0,0,121,347,71969,8129,476,6,0,5392,5386,1022,5582,1022,282,250,561480,652373,0,124171,2334,85851,138752,181570,34237,48700,0,12949,170158,224177,6679,806,5661,...,1,1,1,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,0,1,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0
446934,1,1,3,27,27,2550,0,0,0,0,347,37947,8129,13,1,0,5392,5386,665,7240,306,25,136,339719,409468,165865,124171,677,631235,475795,431730,89207,98464,0,6809,136089,192301,0,203,15860,...,0,0,1,1,0,0,0,1,1,1,0,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0,1,1,0,0,1,0,0,1,0,0,0,0,0,1
94832,0,0,0,27,27,2550,0,0,0,0,347,0,8129,11,0,0,5392,5386,204,6651,16,0,250,56681,138959,165865,124171,9746,274108,100341,150538,89207,98464,0,0,0,0,0,7158,410007,...,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
27932,0,0,3,27,0,2550,0,0,0,0,347,0,8129,34,0,0,5392,5386,258,3438,20,0,250,152307,393908,165865,124171,2543,546925,548183,577573,89207,98464,0,0,100182,93254,0,901,361359,...,0,0,1,1,0,1,1,0,1,1,0,0,0,0,1,1,1,0,0,0,0,1,1,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0
372895,1627,10,36,27,27,2550,41854,0,0,121,347,71969,8129,27,6,0,5392,5386,291,2427,291,282,250,323299,199151,165865,124171,9746,313969,113215,145438,89207,98464,0,12949,170158,224177,6679,7158,2201,...,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176358,0,0,0,27,27,2550,0,0,0,0,347,0,8129,14,0,0,5392,5386,353,3422,61,0,250,4998,257692,165865,124171,9746,1783,7819,334902,89207,98464,0,0,0,0,0,7158,467199,...,0,0,1,1,0,1,1,1,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,0,0,1,0,0,0,0,0,1
317995,0,0,0,27,0,2550,12400,0,0,0,303,0,8129,58,0,0,2365,2364,385,7639,385,0,250,410908,502982,0,124171,9746,488702,528036,534709,89207,98464,0,0,0,0,0,7158,279196,...,0,0,1,1,1,1,1,0,0,1,0,0,0,0,1,1,0,0,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1
392643,1627,10,36,27,27,2550,34380,0,0,121,347,71969,8129,33,6,0,5392,5386,2116,2112,603,282,250,458845,538398,165865,124171,9746,542098,578667,594045,89207,98464,0,12949,170158,224177,6679,7158,374904,...,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0
25295,0,0,0,27,0,2550,9530,0,0,0,347,0,8129,31,0,0,5392,5386,964,6399,953,0,250,128643,232891,0,124171,282,313758,425497,481783,89207,98464,0,0,0,0,0,77,156961,...,0,0,1,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1


### Пакет моделей 1. Обычный датасет

In [249]:
X = shuffled_data_not_balanced.drop(['target'], axis=1)
y = shuffled_data_not_balanced['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

# CatBoost

In [254]:
categorical_features_indices = [X_train.columns.get_loc(col) for col in df_categorical.columns]

In [255]:
def objective(trial):
    # Определение гиперпараметров для оптимизации
    params = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10),
        'loss_function': 'Logloss',
        'eval_metric': 'AUC',
        'random_seed': 42,
        'task_type': 'CPU' 
    }

    # Создание модели CatBoost с текущими гиперпараметрами
    model = CatBoostClassifier(**params)

    # Обучение модели на сбалансированных данных
    model.fit(X_train, y_train, cat_features=categorical_features_indices, verbose=False)

    # Оценка производительности модели на тестовом наборе
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    roc_auc = roc_auc_score(y_test, y_pred_proba)

    return roc_auc

# Создание объекта study и запуск оптимизации
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=25)

# Получение лучших гиперпараметров
best_params = study.best_params
print("Best params:", best_params)

# Создание модели CatBoost с лучшими гиперпараметрами
best_model = CatBoostClassifier(**best_params)

# Обучение модели на сбалансированных данных
best_model.fit(X_train, y_train, cat_features=categorical_features_indices)

# Оценка производительности модели на тестовом наборе
y_pred_proba_best = best_model.predict_proba(X_test)[:, 1]
roc_auc_best = roc_auc_score(y_test, y_pred_proba_best)
print("Best ROC AUC Score:", roc_auc_best)

[I 2024-03-14 04:11:42,354] A new study created in memory with name: no-name-f316a6b5-8da0-4f5a-9f51-0020ddd82e94
<ipython-input-255-3a7d6b14bcc2>:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
<ipython-input-255-3a7d6b14bcc2>:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10),
[I 2024-03-14 04:12:14,246] Trial 0 finished with value: 0.74084113911536 and parameters: {'iterations': 393, 'learning_rate': 0.03192140832723211, 'depth': 10, 'l2_leaf_reg': 0.015822535178055885}. Best is trial 0 with value: 0.74084113911536.
<ipython-input-25

Best params: {'iterations': 737, 'learning_rate': 0.022211908744044193, 'depth': 6, 'l2_leaf_reg': 0.02760089051572016}
0:	learn: 0.6902643	total: 15.3ms	remaining: 11.2s
1:	learn: 0.6873831	total: 27.1ms	remaining: 9.95s
2:	learn: 0.6844134	total: 39.2ms	remaining: 9.59s
3:	learn: 0.6817724	total: 51.9ms	remaining: 9.5s
4:	learn: 0.6794646	total: 64.4ms	remaining: 9.43s
5:	learn: 0.6768925	total: 77.1ms	remaining: 9.4s
6:	learn: 0.6746766	total: 89.2ms	remaining: 9.3s
7:	learn: 0.6725308	total: 101ms	remaining: 9.18s
8:	learn: 0.6705754	total: 113ms	remaining: 9.13s
9:	learn: 0.6686505	total: 126ms	remaining: 9.19s
10:	learn: 0.6665399	total: 139ms	remaining: 9.15s
11:	learn: 0.6647001	total: 151ms	remaining: 9.12s
12:	learn: 0.6629881	total: 164ms	remaining: 9.11s
13:	learn: 0.6612542	total: 176ms	remaining: 9.1s
14:	learn: 0.6595241	total: 188ms	remaining: 9.06s
15:	learn: 0.6579043	total: 200ms	remaining: 9.03s
16:	learn: 0.6564275	total: 213ms	remaining: 9s
17:	learn: 0.6548051	to

In [256]:
# Вычисляем значения FPR и TPR для различных порогов
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba_best)

# Вычисляем значения AUC-ROC для каждого порога
auc_roc_values = []
for threshold in thresholds:
    binary_predictions = (y_pred_proba_best >= threshold).astype(int)
    auc_roc = roc_auc_score(y_test, binary_predictions)
    auc_roc_values.append(auc_roc)

# Находим порог, при котором AUC-ROC максимально
optimal_threshold = thresholds[np.argmax(auc_roc_values)]
print("Optimal threshold:", optimal_threshold)

Optimal threshold: 0.4780811752205957


In [257]:
# Предсказание вероятностей для тестовых данных с помощью модели CatBoost
pred_proba_catboost = best_model.predict_proba(test_data)[:, 1]

# Преобразование вероятностей в бинарные предсказания с использованием оптимального порога
pred_binary_catboost = (pred_proba_catboost >= optimal_threshold).astype(int)

# Создание DataFrame для submission с предсказаниями модели CatBoost
submission_catboost = submission.copy()
submission_catboost["target_prob"] = pred_proba_catboost
submission_catboost["target_bin"] = pred_binary_catboost

In [258]:
submission_catboost['target_bin'].value_counts()

0    127775
1     45658
Name: target_bin, dtype: int64

In [259]:
submission_catboost

,id,target_bin,target_prob
0,3,0,0.276677
1,4,0,0.308192
2,12,0,0.371309
3,16,0,0.199385
4,20,1,0.622193
...,...,...,...
173428,693043,1,0.858162
173429,693044,1,0.757877
173430,693045,1,0.834616
173431,693046,1,0.886315


In [260]:
submission_catboost.to_csv("submission_cb_second_600.csv")